In [72]:
import json

In [73]:
with open('./profile-www.cryptonoter.me.json') as f:
    raw = json.load(f)

In [74]:
profile = raw['profile']
profile.keys()

dict_keys(['nodes', 'timeDeltas', 'startTime', 'endTime', 'samples'])

In [75]:
import numpy as np

# Unit: microseconds
np.average(profile['timeDeltas']), np.std(profile['timeDeltas'])

(198.75716908608186, 318.72477710263485)

In [76]:
# This should be equivalent to "sorting by self time" in chrome gui
sorted(profile['nodes'], key=lambda x: x['hitCount'], reverse=True)[:4]

[{'callFrame': {'columnNumber': -1,
   'functionName': '(idle)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 299699,
  'id': 4},
 {'callFrame': {'columnNumber': -1,
   'functionName': '(program)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 2520,
  'id': 2},
 {'callFrame': {'columnNumber': 29277,
   'functionName': '',
   'lineNumber': 0,
   'scriptId': '19',
   'url': ''},
  'children': [278, 280, 285, 289, 295, 301, 315, 316],
  'hitCount': 123,
  'id': 272,
  'positionTicks': [{'line': 1, 'ticks': 123}]},
 {'callFrame': {'columnNumber': 25413,
   'functionName': '_0x2d0c.lbeIEt',
   'lineNumber': 0,
   'scriptId': '16',
   'url': 'https://www.cryptonoter.me/prototyper-show.js'},
  'children': [11, 12, 13, 14, 15, 16, 17, 18, 19],
  'hitCount': 63,
  'id': 10,
  'positionTicks': [{'line': 1, 'ticks': 63}]}]

In [77]:
result = {}
def calc(node_id):
    cached = result.get(node_id)
    if cached:
        return cached
    
    node = profile['nodes'][node_id - 1]
    child = node.get('children', [])
    ans = node['hitCount']
    for cid in child:
        ans += calc(cid)
    
    result[node_id] = ans
    return ans

calc(1)

302842

In [89]:
sorted(result.items(), key=lambda x: x[1], reverse=True)[:50]

[(1, 302842),
 (4, 299699),
 (2, 2520),
 (5, 147),
 (273, 143),
 (8, 99),
 (245, 99),
 (140, 96),
 (141, 95),
 (148, 90),
 (149, 90),
 (150, 88),
 (153, 85),
 (10, 84),
 (157, 81),
 (158, 81),
 (249, 77),
 (251, 56),
 (159, 52),
 (160, 50),
 (76, 45),
 (49, 41),
 (50, 41),
 (77, 41),
 (3, 32),
 (177, 30),
 (194, 28),
 (204, 26),
 (30, 21),
 (102, 15),
 (103, 15),
 (104, 15),
 (184, 15),
 (105, 14),
 (185, 14),
 (20, 13),
 (257, 13),
 (258, 13),
 (53, 12),
 (89, 10),
 (161, 10),
 (187, 10),
 (265, 10),
 (54, 9),
 (27, 8),
 (31, 8),
 (90, 8),
 (91, 8),
 (92, 8),
 (210, 8)]

In [110]:
def getAllChildren(cid):
    node = profile['nodes'][cid-1]
    child = node.get('children', [])
    ans = list(child)
    for x in child:
        ans += getAllChildren(x)
    return ans
print(result[273])
print(max([(result[x], profile['nodes'][x-1]['callFrame']) for x in getAllChildren(273)], key=lambda x: x[0]))
child = getAllChildren(273) + [273]
print(child)
sum([profile['timeDeltas'][i] for i, x in enumerate(profile['samples']) if x in child])

143
(8, {'lineNumber': 0, 'columnNumber': 16161, 'url': '', 'functionName': 'h.(anonymous function).(anonymous function)', 'scriptId': '19'})
[278, 280, 285, 289, 295, 301, 315, 316, 286, 287, 312, 314, 320, 322, 293, 275, 326, 273]


8591

In [120]:
assert profile['nodes'][1]['callFrame']['functionName'] == '(program)'
print((result[273] + result[5]) / profile['nodes'][1]['hitCount'])

0.11507936507936507


In [45]:
set(x['callFrame']['functionName'] for x in profile['nodes'] if '(' in x['callFrame']['functionName'])
set(x['callFrame']['functionName'] for x in profile['nodes'])

{'',
 '(garbage collector)',
 '(idle)',
 '(program)',
 '(root)',
 'Add',
 'Bd',
 'Ca',
 'Da.addAll',
 'Fd',
 'G',
 'Gd',
 'Ha.D',
 'Hd.a.push',
 'Jc',
 'Kc',
 'LdaNamedProperty',
 'N.N',
 'N.create',
 'Pa',
 'Pd',
 'Qd',
 'S',
 'Sd',
 'StringPrototypeReplace',
 'StringPrototypeSlice',
 'Ua',
 'Ub',
 'X.b.(anonymous function)',
 'Xc',
 'Yc',
 'Z.D',
 'Z.v',
 '_0x2d0c',
 '_0x2d0c.lbeIEt',
 'bd',
 'ee.set',
 'fc',
 'g.(anonymous function).(anonymous function)',
 'h',
 'h.(anonymous function).(anonymous function)',
 'i.(anonymous function).(anonymous function)',
 'ma',
 'mc',
 'pc',
 'pc.send',
 'ra',
 'rc',
 'y',
 'z',
 'zc',
 'zf.fd'}